In [1]:
import torch
from torch import nn
import torch.optim as optim
import numpy as np
import random

In [2]:
class NN2048(nn.Module):
    def __init__(self, input_size=16, filter1=512, filter2=4096, drop_prob=0.):
        super(NN2048, self).__init__()
        self.conv_a = nn.Conv2d(in_channels=input_size, out_channels=filter1, kernel_size=(2,1), padding=0)
        self.conv_b = nn.Conv2d(in_channels=input_size, out_channels=filter1, kernel_size=(1,2), padding=0)
        self.conv_c = nn.Conv2d(in_channels=input_size, out_channels=filter1, kernel_size=(2,2), padding=0)
        
        self.conv_aa = nn.Conv2d(in_channels=filter1, out_channels=filter2, kernel_size=(2,1), padding=0)
        self.conv_ab = nn.Conv2d(in_channels=filter1, out_channels=filter2, kernel_size=(1,2), padding=0)
        self.conv_ba = nn.Conv2d(in_channels=filter1, out_channels=filter2, kernel_size=(2,1), padding=0)
        self.conv_bb = nn.Conv2d(in_channels=filter1, out_channels=filter2, kernel_size=(1,2), padding=0)
        self.conv_cc = nn.Conv2d(in_channels=filter1, out_channels=filter2, kernel_size=(2,2), padding=0)
        
        self.relu = nn.ReLU()
        self.W_aa = nn.Linear(filter2 * 8, 1)
        self.W_ab = nn.Linear(filter2 * 9, 1)
        self.W_ba = nn.Linear(filter2 * 9, 1)
        self.W_bb = nn.Linear(filter2 * 8, 1)
        self.W_cc = nn.Linear(filter2 * 4, 1)

    def flatten(self, x):
        N = x.size()[0]
        return x.view(N, -1)
        
    def forward(self, x):
        x = x.float()
        a = self.relu(self.conv_a(x))
        b = self.relu(self.conv_b(x))
        c = self.relu(self.conv_c(x))
        aa = self.flatten(self.relu(self.conv_aa(a)))
        ab = self.flatten(self.relu(self.conv_ab(a)))
        ba = self.flatten(self.relu(self.conv_ba(b)))
        bb = self.flatten(self.relu(self.conv_bb(b)))
        cc = self.flatten(self.relu(self.conv_cc(c)))
        out = self.W_aa(aa) + self.W_ab(ab) + self.W_ba(ba) + self.W_bb(bb) + self.W_cc(cc)
        return out

In [3]:
def make_input(grid):
    r = np.zeros(shape=(16, 4, 4))
    for i in range(4):
        for j in range(4):
            r[grid[i, j],i, j]=1
    return r

def add_two(mat):
    indexs=np.argwhere(mat==0)
    index=np.random.randint(0,len(indexs))
    mat[tuple(indexs[index])] = 1
    return mat

In [4]:
singleScore=[0,0,4,16,48,128,320,768,1792,4096,9216,20480,45056,98304,212992,458752,983040]
moveDict=np.load('move.npy')

def move(list):
    return moveDict[list[0],list[1],list[2],list[3],:]

def lookup(x):
    return singleScore[x]

lookup = np.vectorize(lookup)

def getScore(matrix):
    return np.sum(lookup(matrix))

def getMove(grid):
    board_list = []
    for i in range(4):
        newGrid=moveGrid(grid, i)
        if not isSame(grid,newGrid):
            board_list.append((newGrid, i, getScore(newGrid)))
    return board_list
        
def moveGrid(grid,i):
    # new=np.zeros((4,4),dtype=np.int)
    new = None
    if i==0:
        # move up
        grid=np.transpose(grid)
        new = np.stack([move(grid[row,:]) for row in range(4)], axis = 0).astype(int).T
    elif i==1:
        # move left
        new = np.stack([move(grid[row,:]) for row in range(4)], axis = 0).astype(int)
    elif i==2:
        # move down
        grid=np.transpose(grid)
        new = np.stack([np.flip(move(np.flip(grid[row,:]))) for row in range(4)], axis = 0).astype(int).T
    elif i==3:
        # move right
        new = np.stack([np.flip(move(np.flip(grid[row,:]))) for row in range(4)], axis = 0).astype(int)
    return new

def isSame(grid1,grid2):
    return np.all(grid1==grid2)

In [5]:
def Vchange(grid, v):
    g0 = grid
    g1 = g0[:,::-1,:]
    g2 = g0[:,:,::-1]
    g3 = g2[:,::-1,:]
    r0 = grid.swapaxes(1,2)
    r1 = r0[:,::-1,:]
    r2 = r0[:,:,::-1]
    r3 = r2[:,::-1,:]
    xtrain = np.array([g0,g1,g2,g3,r0,r1,r2,r3])
    ytrain = np.array([v]*8)
    return xtrain, ytrain

def gen_sample_and_learn(model, optimizer, loss_fn, is_train = False, explorationProb=0.1):
    model.eval()
    game_len = 0
    game_score = 0
    last_grid1 = np.zeros((4,4),dtype=np.int)
    last_grid1 = add_two(last_grid1)
    last_grid2 = make_input(last_grid1)
    last_loss = 0

    while True:
        grid_array = add_two(last_grid1)
        board_list = getMove(grid_array)
        if board_list:
            boards = np.array([make_input(g) for g,m,s in board_list])
            p = model(torch.from_numpy(boards).cuda()).flatten().detach()        
            game_len += 1
            best_v = None
            for i, (g,m,s) in enumerate(board_list):
                v = (s - game_score) + p[i].item()
                if best_v is None or v > best_v:
                    best_v = v
                    best_score = s
                    best_grid1 = board_list[i][0]
                    best_grid2 = boards[i]
                    
        else:
            best_v = 0
            best_grid1 = None
            best_grid2 = None
            
        if is_train:
            x, y = Vchange(last_grid2, best_v)
            x = torch.from_numpy(x).cuda()
            y = torch.from_numpy(y).unsqueeze(dim=1).cuda().float()
            model.train()
            optimizer.zero_grad()
            pred = model(x)
            loss = loss_fn(pred, y)
            last_loss = loss.item()
            loss.backward()
#             nn.utils.clip_grad_norm_(model.parameters(), 10.0) #
            optimizer.step()
            model.eval()
                
        if not board_list:
            break
            
        # gibbs sampling or espilon-greedy
        if is_train and random.random() < explorationProb:
            idx = random.randint(0, len(board_list) - 1)
            game_score = board_list[idx][2]
            last_grid1 = board_list[idx][0]
            last_grid2 = boards[idx]
        else:
            game_score = best_score
            last_grid1 = best_grid1
            last_grid2 = best_grid2
        
    return game_len, 2**grid_array.max(), game_score, last_loss

In [6]:
lr = 2e-4
weight_decay = 0
beta1 = 0.9

model = NN2048().cuda()
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay, betas=(beta1, 0.999))
loss=nn.MSELoss()

In [7]:
import os
experiment_dir = "model"

def save_model(state, filename='model.pth.tar'):
    filename = os.path.join(experiment_dir, filename)
    torch.save(state, filename)

In [8]:
def load_model(model, optimizer, checkpoint_path):
    ckpt_dict = torch.load(checkpoint_path, map_location="cuda:0")

    model.load_state_dict(ckpt_dict['state_dict'])
    optimizer.load_state_dict(ckpt_dict['optimizer'])
    epoch = ckpt_dict['epoch']
    running_mean = ckpt_dict['running_mean']
    return model, optimizer, epoch

# model, optimizer, epoch, running_mean = load_model(model, optimizer, "model/model7_2_1000.pth.tar")

In [ ]:
num_epochs = 3000

def train(model, optimizer, loss, epoch = 0, running_mean = 2048):
    ls = [1024] * 10
    while epoch != num_epochs:
        epoch += 1
        game_len, max_score, game_score, last_loss = gen_sample_and_learn(model, optimizer, loss, True, 0)
        print ('Game # ', epoch, 'Game length ', game_len, 'Max score ', max_score, 'Game score ', game_score, flush=True)
        ls.pop(0)
        ls.append(max_score)
        if sum(ls) / 10 > running_mean:
            running_mean = sum(ls) / 10
            filename = "model7_2_"+str(epoch // 100)+".pth.tar"
            save_model({
                'epoch': epoch,
                'running_mean': running_mean,
                'state_dict': model.cpu().state_dict(),
                'optimizer': optimizer.state_dict(),
            }, filename)
            model.cuda()
    
    
train(model, optimizer, loss)

Game #  1 Game length  187 Max score  128 Game score  1740
Game #  2 Game length  135 Max score  64 Game score  1068
Game #  3 Game length  226 Max score  128 Game score  2264
Game #  4 Game length  158 Max score  128 Game score  1456
Game #  5 Game length  176 Max score  128 Game score  1612
Game #  6 Game length  174 Max score  128 Game score  1564
Game #  7 Game length  207 Max score  128 Game score  1932
Game #  8 Game length  311 Max score  256 Game score  3532
Game #  9 Game length  237 Max score  256 Game score  2608
Game #  10 Game length  200 Max score  128 Game score  1868
Game #  11 Game length  257 Max score  256 Game score  2944
Game #  12 Game length  239 Max score  256 Game score  2800
Game #  13 Game length  280 Max score  256 Game score  3072
Game #  14 Game length  171 Max score  128 Game score  1572
Game #  15 Game length  230 Max score  256 Game score  2536
Game #  16 Game length  392 Max score  512 Game score  5296
Game #  17 Game length  219 Max score  128 Game sc

Game #  136 Game length  1047 Max score  1024 Game score  16524
Game #  137 Game length  1005 Max score  1024 Game score  16112
Game #  138 Game length  843 Max score  1024 Game score  13100
Game #  139 Game length  1038 Max score  1024 Game score  16456
Game #  140 Game length  973 Max score  1024 Game score  15640
Game #  141 Game length  1041 Max score  1024 Game score  16472
Game #  142 Game length  340 Max score  256 Game score  3936
Game #  143 Game length  953 Max score  1024 Game score  15080
Game #  144 Game length  1005 Max score  1024 Game score  16000
Game #  145 Game length  527 Max score  512 Game score  7240
Game #  146 Game length  529 Max score  512 Game score  7248
Game #  147 Game length  1049 Max score  1024 Game score  16552
Game #  148 Game length  792 Max score  1024 Game score  12444
Game #  149 Game length  465 Max score  512 Game score  6464
Game #  150 Game length  288 Max score  256 Game score  3276
Game #  151 Game length  528 Max score  512 Game score  724

Game #  267 Game length  1736 Max score  2048 Game score  31772
Game #  268 Game length  539 Max score  512 Game score  7340
Game #  269 Game length  918 Max score  1024 Game score  14712
Game #  270 Game length  764 Max score  1024 Game score  12140
Game #  271 Game length  546 Max score  512 Game score  7400
Game #  272 Game length  1463 Max score  2048 Game score  26332
Game #  273 Game length  1789 Max score  2048 Game score  32644
Game #  274 Game length  902 Max score  1024 Game score  14072
Game #  275 Game length  563 Max score  512 Game score  7956
Game #  276 Game length  1041 Max score  1024 Game score  16496
Game #  277 Game length  1711 Max score  2048 Game score  30356
Game #  278 Game length  980 Max score  1024 Game score  15712
Game #  279 Game length  792 Max score  1024 Game score  12416
Game #  280 Game length  788 Max score  1024 Game score  12384
Game #  281 Game length  1537 Max score  2048 Game score  27600
Game #  282 Game length  435 Max score  512 Game score 

Game #  398 Game length  1836 Max score  2048 Game score  33516
Game #  399 Game length  2182 Max score  2048 Game score  38684
Game #  400 Game length  1938 Max score  2048 Game score  35164
Game #  401 Game length  641 Max score  512 Game score  8912
Game #  402 Game length  1111 Max score  1024 Game score  17372
Game #  403 Game length  860 Max score  1024 Game score  13252
Game #  404 Game length  1235 Max score  1024 Game score  20828
Game #  405 Game length  1852 Max score  2048 Game score  33348
Game #  406 Game length  615 Max score  512 Game score  8452
Game #  407 Game length  2018 Max score  2048 Game score  36348
Game #  408 Game length  1969 Max score  2048 Game score  35856
Game #  409 Game length  1557 Max score  2048 Game score  27768
Game #  410 Game length  1050 Max score  1024 Game score  16616
Game #  411 Game length  1173 Max score  1024 Game score  18328
Game #  412 Game length  1015 Max score  1024 Game score  16092
Game #  413 Game length  2956 Max score  4096 G

Game #  527 Game length  1927 Max score  2048 Game score  35068
Game #  528 Game length  1027 Max score  1024 Game score  16348
Game #  529 Game length  2565 Max score  4096 Game score  52224
Game #  530 Game length  2581 Max score  4096 Game score  52332
Game #  531 Game length  944 Max score  1024 Game score  15020
Game #  532 Game length  2023 Max score  2048 Game score  36528
Game #  533 Game length  1812 Max score  2048 Game score  32880
Game #  534 Game length  1840 Max score  2048 Game score  33176
Game #  535 Game length  1077 Max score  1024 Game score  16976
Game #  536 Game length  1065 Max score  1024 Game score  16752
Game #  537 Game length  1479 Max score  2048 Game score  26564
Game #  538 Game length  670 Max score  512 Game score  9164
Game #  539 Game length  1921 Max score  2048 Game score  34472
Game #  540 Game length  1747 Max score  2048 Game score  32092
Game #  541 Game length  2058 Max score  2048 Game score  36920
Game #  542 Game length  1340 Max score  102

Game #  656 Game length  1245 Max score  1024 Game score  19240
Game #  657 Game length  2309 Max score  2048 Game score  40984
Game #  658 Game length  2128 Max score  2048 Game score  37820
Game #  659 Game length  2794 Max score  4096 Game score  56944
Game #  660 Game length  1051 Max score  1024 Game score  16572
Game #  661 Game length  1638 Max score  2048 Game score  28764
Game #  662 Game length  1877 Max score  2048 Game score  33688
Game #  663 Game length  1958 Max score  2048 Game score  35416
Game #  664 Game length  537 Max score  512 Game score  7304
Game #  665 Game length  1692 Max score  2048 Game score  29640
Game #  666 Game length  898 Max score  1024 Game score  13524
Game #  667 Game length  1318 Max score  2048 Game score  23804
Game #  668 Game length  2073 Max score  2048 Game score  37048
Game #  669 Game length  2096 Max score  2048 Game score  37288
Game #  670 Game length  3120 Max score  4096 Game score  61864
Game #  671 Game length  2021 Max score  204

Game #  785 Game length  899 Max score  1024 Game score  13624
Game #  786 Game length  1887 Max score  2048 Game score  33752
Game #  787 Game length  1573 Max score  2048 Game score  27900
Game #  788 Game length  3364 Max score  4096 Game score  68940
Game #  789 Game length  976 Max score  1024 Game score  15440
Game #  790 Game length  1877 Max score  2048 Game score  34420
Game #  791 Game length  1882 Max score  2048 Game score  34444
Game #  792 Game length  1578 Max score  2048 Game score  27940
Game #  793 Game length  3088 Max score  4096 Game score  61528
Game #  794 Game length  2085 Max score  2048 Game score  37208
Game #  795 Game length  3013 Max score  4096 Game score  60684
Game #  796 Game length  1997 Max score  2048 Game score  36152
Game #  797 Game length  2250 Max score  2048 Game score  41004
Game #  798 Game length  2046 Max score  2048 Game score  36748
Game #  799 Game length  3122 Max score  4096 Game score  61900
Game #  800 Game length  2112 Max score  2

Game #  914 Game length  756 Max score  512 Game score  10200
Game #  915 Game length  817 Max score  1024 Game score  12648
Game #  916 Game length  1178 Max score  1024 Game score  18364
Game #  917 Game length  1738 Max score  2048 Game score  31612
Game #  918 Game length  1481 Max score  2048 Game score  26880
Game #  919 Game length  1557 Max score  2048 Game score  27768
Game #  920 Game length  2869 Max score  4096 Game score  57816
Game #  921 Game length  2128 Max score  2048 Game score  37752
Game #  922 Game length  2055 Max score  2048 Game score  36892
Game #  923 Game length  1208 Max score  1024 Game score  18680
Game #  924 Game length  3009 Max score  4096 Game score  60232
Game #  925 Game length  1819 Max score  2048 Game score  32932
Game #  926 Game length  672 Max score  512 Game score  9176
Game #  927 Game length  1694 Max score  2048 Game score  30156
Game #  928 Game length  2604 Max score  4096 Game score  52612
Game #  929 Game length  2054 Max score  2048 

In [10]:
num_epochs = 100

def test(model):
    epoch = 0
    while epoch != num_epochs:
        epoch += 1
        game_len, max_score, game_score, last_loss = gen_sample_and_learn(model, None, None, False)
        print ('Game # ', epoch, 'Game length ', game_len, 'Max score ', max_score, 'Game score ', game_score, flush=True)

test(model)

Game #  1 Game length  831 Max score  1024 Game score  12940
Game #  2 Game length  908 Max score  1024 Game score  14636
Game #  3 Game length  924 Max score  1024 Game score  14744
Game #  4 Game length  1028 Max score  1024 Game score  16352
Game #  5 Game length  929 Max score  1024 Game score  14808
Game #  6 Game length  551 Max score  512 Game score  7452
Game #  7 Game length  547 Max score  512 Game score  7392
Game #  8 Game length  1010 Max score  1024 Game score  16028
Game #  9 Game length  534 Max score  512 Game score  7288
Game #  10 Game length  693 Max score  1024 Game score  10872
Game #  11 Game length  1033 Max score  1024 Game score  16360
Game #  12 Game length  800 Max score  1024 Game score  12440
Game #  13 Game length  562 Max score  512 Game score  7532
Game #  14 Game length  1316 Max score  2048 Game score  23796
Game #  15 Game length  795 Max score  1024 Game score  12460
Game #  16 Game length  1029 Max score  1024 Game score  16384
Game #  17 Game leng